In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.autograd import Variable

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(3,6,5)
        self.conv2=nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(5*5*16,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
        
    def forward(self,x):
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        x=x.view(-1,self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    
    def num_flat_features(self,x):
        size=x.size()[1:]
        num_features=1
        for s in size:
            num_features*=s
        return num_features    
        
net=Net()
net

Net(
  (conv1): Conv2d (3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d (6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120)
  (fc2): Linear(in_features=120, out_features=84)
  (fc3): Linear(in_features=84, out_features=10)
)

In [3]:
params=list(net.parameters())
print(len(params))
print(params[0].size())
input=Variable(torch.randn(1,3,32,32)) # 1 batch, 3 channel, 32x32 size
out=net(input)
out

10
torch.Size([6, 3, 5, 5])


Variable containing:
1.00000e-02 *
  6.6231 -2.5589 -9.4124  0.6895  1.9218  8.0875  0.8771 -5.3526  1.1202 -7.6638
[torch.FloatTensor of size 1x10]

In [4]:
target=Variable(torch.arange(0,10)) #[0,10) not include 10
criterion=nn.MSELoss()
loss=criterion(out,target)
loss

Variable containing:
 28.6298
[torch.FloatTensor of size 1]

In [5]:
print(loss.grad_fn)

In [6]:
import torchvision
import torchvision.transforms as transforms


# torchvision数据集的输出是在[0, 1]范围内的PILImage图片。
# 我们此处使用归一化的方法将其转化为Tensor，数据范围为[-1, 1]

transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                             ])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, 
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, 
                                          shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [7]:
import matplotlib.pyplot as plt
import numpy as np
def imshow(img):
    img=img/2+0.5
    npimg=img.numpy()
    plt.imshow(np.transpose(npimg)) # transpose indices, eg (2,0,1), default exchange first two dimension

dataiter=iter(trainloader)
images,labels=dataiter.next()
imshow(torchvision.utils.make_grid(images))

print(' '.join('%5s'%classes[labels[j]] for j in range(4)))

 ship plane   car horse


In [8]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(net.parameters(),lr=0.001,momentum=0.9)

In [9]:
for epoch in range(2):
    running_loss=0.0
    for idx,data in enumerate(trainloader,0):
        img,lab=data
        img,lab=Variable(img),Variable(lab) # wrap into variable
        # Variable是Tensor的一个wrapper, 一个是保存weights的Tensor，一个是保存grad的Variable。Variable的一些运算，
        # 实际上就是里面的Tensor的运算, Variable默认代表的是里面存储的Tensor（weights）
        # 更新的时候要用Tensor更新. Variable更多是用在feedforward中的，因为feedforward是需要记住各个Tensor之间联系的，
        # 这样，才能正确的bp。Tensor不会记录路径。

        # general
        optimizer.zero_grad() # zero the parameter gradients
        out=net(img) # forward
        loss=criterion(out,lab) # loss function
        loss.backward() # back propagation
        # when execute backward for the second time, using retain_variables=True
        # Further reading: Trying to backward through the graph second time, but the buffers have already been 
        # freed. Please specify retain_variables=True when calling backward for the first time
        optimizer.step() # update params using SGD
        
        running_loss+=loss.data[0]
        if idx % 2000 == 1999: # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch+1, idx+1, running_loss / 2000))
            running_loss = 0.0
        # mini-batches size 2k

print("finish training")



[1,  2000] loss: 2.214
[1,  4000] loss: 1.867
[1,  6000] loss: 1.717
[1,  8000] loss: 1.607
[1, 10000] loss: 1.543
[1, 12000] loss: 1.497
[2,  2000] loss: 1.444
[2,  4000] loss: 1.367
[2,  6000] loss: 1.372
[2,  8000] loss: 1.333
[2, 10000] loss: 1.309
[2, 12000] loss: 1.263
finish training


In [10]:
correct=0
total=0
for data in testloader:
    img,lab=data
    out=net(Variable(img))
    _,predict=torch.max(out.data,1) # out.data是一个4x10张量，max函数会将每一行的最大的那一列的值和序号各自组成一个一维张量返回，
                                    # 第一个是值的张量，第二个是序号的张量
    total+=lab.size(0)
    correct+=(predict==lab).sum()
    
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 55 %
